In [ ]:
import sys
import os

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
from forcateri.model.dartsmodels.dartstcnmodel import DartsTCNModel
from forcateri.baltbestapi.baltbestaggregatedapidata import BaltBestAggregatedAPIData
import pandas as pd
from forcateri.data.dataprovider import DataProvider, SeriesRole
from darts.models import TCNModel
from darts.utils.likelihood_models import QuantileRegression
from forcateri.data.timeseries import TimeSeries

In [ ]:
%load_ext autoreload
%autoreload 2 

In [ ]:
# baltbest = BaltBestAggregatedAPIData(
#     name='test', 
#     url="baltbest_url", 
#     local_copy="/home/user/DFKI/forcateri/_data/showcase_data.csv",
#     target = 'q_hca',
#     group_col = 'room_id',
#     time_col = 'datetime',
#     known = 'temperature_outdoor_avg',
#     observed = ['temperature_1_max', 'temperature_2_max','temperature_room_avg'],)
baltbest = BaltBestAggregatedAPIData()

In [ ]:
import logging

# Get the logger you used in your TimeSeries class
logger = logging.getLogger("forcateri.data.timeseries")
logger.setLevel(logging.INFO)  # or logging.DEBUG for more verbosity

# Create handler if it doesn't exist
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# Optional: prevent propagation to avoid double logs
logger.propagate = False


In [ ]:

#baltbest.get_data()


In [ ]:
roles = {
    'q_hca': SeriesRole.TARGET, 
    'temperature_outdoor_avg':SeriesRole.KNOWN, 
    'temperature_1_max':SeriesRole.OBSERVED, 
    'temperature_2_max':SeriesRole.OBSERVED,
    'temperature_room_avg':SeriesRole.OBSERVED,}
#['temperature_1_max', 'temperature_2_max','temperature_room_avg']
# roles = {
#     'delta': SeriesRole.TARGET, 
#     'outside_temp':SeriesRole.KNOWN, 
#     'max_temperature_1':SeriesRole.OBSERVED, 
#     'max_temperature_1':SeriesRole.OBSERVED,
#     'room_temperature':SeriesRole.OBSERVED,}

In [ ]:
start = pd.Timestamp(2021, 1, 3, 4,tz=0)
end = pd.Timestamp(2021, 1, 8, 4,tz=0)
dataprovider = DataProvider(data_sources=[baltbest], roles=roles)

In [ ]:
train = dataprovider.get_train_set()
val = dataprovider.get_val_set()
test = dataprovider.get_test_set()

In [ ]:
train[0].target.data

In [ ]:
# import os
# #os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# quantiles = [0.1, 0.5, 0.9]
# dartstcn = DartsTCNModel( quantiles=quantiles)
# dartstcn.fit(train_data = train, val_data=val)

In [ ]:
num_forecast_steps = 5
#num_monte_carlo_samples = 50 # A sufficient number of samples to approximate the quantiles
dartstcn = DartsTCNModel.load( "/home/user/DFKI/forcateri/_data/dartstcn.pt")
quantiles = [0.1, 0.5, 0.9] # Define the quantiles you want to predict
dartstcn.quantiles = quantiles
#Here prediction is darts timeseries
prediction = dartstcn.predict(
    data=val, # Pass the series used for prediction context
    predict_likelihood_parameters = True,
    n=num_forecast_steps,
)

In [ ]:
from IPython.display import display, Markdown
def mprint(s): display(Markdown(s))

In [ ]:
prediction

In [ ]:
prediction_ts = dartstcn.to_time_series(prediction)

In [ ]:
dum = prediction_ts[0].data
#Need to fix timestamp and offset

In [ ]:
import pandas as pd
import numpy as np

# Offsets (timedeltas)
offsets = pd.to_timedelta(['1:00:00', '2:00:00', '3:00:00', '4:00:00', '5:00:00'])

# Multiple timestamps per offset
time_stamps = pd.to_datetime([
    '2020-08-29 03:00:00',
    '2020-08-29 04:00:00',
    '2020-08-29 05:00:00',
])

# Cartesian product for MultiIndex rows
offset_repeated = np.repeat(offsets.values, len(time_stamps))
time_stamp_tiled = np.tile(time_stamps.values, len(offsets))

multi_idx = pd.MultiIndex.from_arrays(
    [offset_repeated, time_stamp_tiled],
    names=['offset', 'time_stamp']
)

N = len(multi_idx)

# Generate base data
base = np.random.normal(loc=0, scale=1, size=N)
q_01 = base - 1.0
q_05 = base
q_09 = base + 1.0

data = np.vstack([q_01, q_05, q_09]).T  # shape (N, 3)

# Define MultiIndex for columns: level 0 = 'feature' name, level 1 = representation
col_idx = pd.MultiIndex.from_product(
    [['q_hca'], [0.1, 0.5, 0.9]],
    names=['feature', 'representation']
)

df_synthetic = pd.DataFrame(data, index=multi_idx, columns=col_idx)

df_synthetic

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
for offset_value, group_df in df_synthetic.groupby(level='offset'):
    ax = group_df.droplevel('offset').plot(
        marker='o', 
        title=f'Offset: {offset_value}', 
        xlabel='time_stamp', 
        ylabel='Value'
    )
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df_plot = df_synthetic.reset_index()
n_offsets = len(offsets)

# Setup subplots: 2 columns, adjust rows automatically
ncols = 2
nrows = (n_offsets + 1) // ncols

fig, axes = plt.subplots(nrows, ncols, figsize=(14, 4*nrows), sharey=True)
axes = axes.flatten()

for i, offset in enumerate(offsets):
    ax = axes[i]
    df_sub = df_plot[df_plot['offset'] == offset]

    # Plotting feature columns: assuming MultiIndex columns like ('feature', 0.1), ('feature', 0.5), etc.
    # We'll plot each feature (the second level of columns)
    for val in df_synthetic.columns.levels[1]:
        sns.lineplot(
            x='time_stamp', 
            y=('feature', val), 
            data=df_sub, 
            marker='o', 
            label=f'feature {val}',
            ax=ax
        )

    ax.set_title(f'Offset: {offset}')
    ax.set_xlabel('Time Stamp')
    ax.set_ylabel('Value')
    ax.tick_params(axis='x', rotation=45)
    ax.legend()

# Remove unused subplots (if any)
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()